<a href="https://colab.research.google.com/github/casangi/ngcasa/blob/master/docs/ngcasa_calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calibration

Radio interferometry data analysis applications and algorithms may be assembled from CNGI and ngCASA building blocks. A user may choose to implement their own analysis scripts, use a pre-packaged task similar to those in current CASA or embed ngCASA and CNGI methods in a production pipeline DAG.

Note : The following examples represent preliminary design ideas that illustrate how ngCASA science applications may be assembled. The API and usage example scripts will change as the details are refined. The current focus is to evaluate whether the CNGI and ngCASA functional design and infrastructure adequately addresses algorithmic needs and how it may be best leveraged for scaleable high performance computing. Questions raised via this initial exercise will guide the design of [future prototypes](https://ngcasa.readthedocs.io/en/latest/ngcasa_development.html#List-of-Future-Prototypes), continued evaluation the chosen infrastructure, and the final function hierarchy and API definition. 


[Edit this notebook in colab](https://colab.research.google.com/github/casangi/ngcasa/blob/master/docs/ngcasa_flagging.ipynb)   : Please note that these examples are currently notional, and not yet runnable in the colab!

##applycal
The following example outlines the basic high-level form of the traditional CASA applycal task, and assumes that a set of calibration tables and a cal library describing how they should be applied is available.

In [0]:
# Construct a select vis dataset
vis_dataset = cngi.dio.read_vis(visname,selpars)

# Gather calibration to apply and parse it (cal library)
# NB: ngcasa.calibration.configcal not yet implemented in the api
# TBD: may be tuned to vis selection from selpars, to trim whole calibration 
#      ensemble to only relevant parts in time, freq, etc.?
cal_ensemble = ngcasa.calibration.configcal(callibrary)

# Apply the calibration to data, forming corrected data
ngcasa.calibration.apply(vis_dataset, 
                         cal_ensemble, 
                         use_arr_name='data', 
                         out_arr_name='corrected_data')

# Write calibrated data to disk
cngi.write_zarr(vis_dataset)

## gaincal
The following example outlines the basic high-level form of gain calibration solving, wherein prior calibration is pre-applied to data and model in a manner consistent with the Measurement Equation.

In [0]:
# Construct a select vis dataset
vis_dataset = cngi.dio.read_vis(visname,selpars)

# Predict model visibilities (in lsrk frame) from a component list, and write to the DATA array in the XDS
cngi.vis.regridspw(vis_dataset)  # Convert from topo to lsrk
ngcasa.imaging.predict_modelvis_component(vis_dataset, component_list, arr_name='model') # De-grid and predict model
cngi.vis.regridspw(vis_dataset)  # Convert from lsrk back to data frame (topo)

# Gather calibration to apply and parse it (cal library)
# NB: ngcasa.calibration.configcal not yet implemented in the api
# NB: may be tuned to vis selection from selpars, to trim whole calibration 
#      ensemble to only relevant parts in time, freq, etc.?
cal_ensemble = ngcasa.calibration.configcal(callibrary)

# Pre-apply the prior calibration to data and model, forming corrected data
# NB: ngcasa.calibration.preapply not yet implemented in the api
# NB: solpars knows which term we are solving for and so informs where to
#     partition pre-applies to upstream (model) and downstream (data) parts
ngcasa.calibration.preapply(vis_dataset, 
                            cal_ensemble,
                            solpars,
                            use_arr_name=['data','model'], 
                            out_arr_name=['corrected_data','corrupted_model')

# Do chan-averaging on original dataset
# NB:  Should be specific to 'corrected_data' and 'corrupted_model'
vis_dataset_chan_avg = cngi.vis.chanaverage(vis_dataset)

# Do time-averaging to solint
# TBD: How is averaging duration specified?
# NB:  Should be specific to 'corrected_data' and 'corrupted_model'
vis_dataset_time_avg = cngi.vis.timeaverage(vis_dataset)

# Solve for gains ('solpars' encodes the solution type)
cal_dataset_gaincal = ngcasa.calibration.solve(vis_dataset_chan_avg, solpars, use_arr_name='corrected_data')

# Write gain solution to disk
cngi.write_zarr(cal_dataset_gaincal)



## bandpass w/ OTF gaincal
The following example outlines the traditional CASA bandpass task, with an additional feature of on-the-fly time-dependent gain calibration to ensure coherence for time-averaging to the bandpass solution interval. 

In [0]:
# Construct a select vis dataset
vis_dataset = cngi.dio.read_vis(visname,selpars)

# Predict model visibilities (in lsrk frame) from a component list, and write to the DATA array in the XDS
cngi.vis.regridspw(vis_dataset)  # Convert from topo to lsrk
ngcasa.imaging.predict_modelvis_component(vis_dataset, component_list, arr_name='model') # De-grid and predict model
cngi.vis.regridspw(vis_dataset)  # Convert from lsrk back to data frame (topo)

# Gather calibration to apply and parse it (cal library)
# NB: ngcasa.calibration.configcal not yet implemented in the api
# NB: may be tuned to vis selection from selpars, to trim whole calibration 
#      ensemble to only relevant parts in time, freq, etc.?
cal_ensemble = ngcasa.calibration.configcal(callibrary)

# Pre-apply the prior calibration to data and model, forming corrected data
# NB: ngcasa.calibration.preapply not yet implemented in the api
# NB: solparsG informs where to partition pre-applies to upstream (model) 
#     and downstream (data) parts
ngcasa.calibration.preapply(vis_dataset, 
                            cal_ensemble,
                            solparsG,
                            use_arr_name=['data','model'], 
                            out_arr_name=['corrected_data','corrupted_model'])

# Do chan-averaging on pre-calibrated dataset
# NB:  Should be specific to 'corrected_data' and 'corrupted_model'
vis_dataset_chan_avg = cngi.vis.chanaverage(vis_dataset)

# Do time-averaging to _G_ solint
# TBD: How is _G_ averaging duration specified?
# NB:  Should be specific to 'corrected_data' and 'corrupted_model'
vis_dataset_chantime_avg = cngi.vis.timeaverage(vis_dataset_chan_avg)

# Solve for temporaray time-dep gains ('solpars' encodes the solution type)
# NB: solparsG describes the G solve
cal_dataset_gaincal = ngcasa.calibration.solve(vis_dataset_chantime_avg, 
                                               solparsG, 
                                               use_arr_name=['corrected_data',
                                                             'corrupted_model')

# Pre-apply prior calibration and temporary gain solution to data
# NB: starting from original data, because we need it w/out chanaverage
# NB: solparsB informs where to partition pre-applies to upstream (model) 
#     and downstream (data) parts
ngcasa.calibration.preapply(vis_dataset, 
                            cal_ensemble+cal_dataset_gaincal,
                            solparsB,
                            use_arr_name=['data','model'], 
                            out_arr_name=['corrected_data','corrupted_model'])

# Do PARTIAL chan-averaging on original dataset, if requested
# Q: How to specify partial channel averaging (for decimated bandpass)
# NB:  Should be specific to 'corrected_data' and 'corrupted_model'
vis_dataset_chan_avg = cngi.vis.chanaverage(vis_dataset)

# Do time-averaging to _B_ solint
# TBD: How is _B_ soling averaging duration specified?
# NB:  Should be specific to 'corrected_data' and 'corrupted_model'
vis_dataset_chantime_avg = cngi.vis.timeaverage(vis_dataset_chan_avg)

# Solve for bandpass ('solpars' encodes the solution type)
# NB: solpars[0] describes the B solve
cal_dataset_bandpass = ngcasa.calibration.solve(vis_dataset_chantime_avg, 
                                                solparsB, 
                                                use_arr_name=['corrected_data',
                                                              'corrupted_model')

# Write bandpass solution to disk
cngi.write_zarr(cal_dataset_bandpass)
